In [7]:
import numpy as np
import pandas as pd

Manual implementation of a levenshtein function

From
https://rosettacode.org/wiki/Levenshtein_distance#Memoized_recursion
and 
https://www.datacamp.com/community/tutorials/fuzzy-string-python
and 


In [13]:
from functools import lru_cache
@lru_cache(maxsize=4095)
def ld(s, t):
    if not s: return len(t)
    if not t: return len(s)
    if s[0] == t[0]: return ld(s[1:], t[1:])
    l1 = ld(s, t[1:])
    l2 = ld(s[1:], t)
    l3 = ld(s[1:], t[1:])
    return 1 + min(l1, l2, l3)
print( ld("kitten","sitting"))
print (ld("rosettacode","raisethysword") )
print (ld("juan","Juan c"))

3
8
3


In [14]:
%%timeit -n1
ld("juan carlos E","juan  carlos")


The slowest run took 307.58 times longer than the fastest. This could mean that an intermediate result is being cached.
20.6 µs ± 49.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
l=["John Smith","Joe Blogger","Peter Woods","Michael Collins","John Smith","Smith, John","John Smitt"]
lk=list(range(1,len(l)+1))
def pre(s):
    return str.lower(s)
lpre=[pre(s) for s in l]
lpre

['john smith',
 'joe blogger',
 'peter woods',
 'michael collins',
 'john smith',
 'smith, john',
 'john smitt']

In [22]:
from itertools import combinations
item_combinations=list(combinations(lpre, 2))
print(len(item_combinations), "combinations to compute. Careful if this number is large, it may take a long time")

21 combinations to compute. Careful if this number is large, it may take a long time


In [25]:
matches = []
for x in item_combinations:
    d = ld(x[0],x[1])
    r=(len(x[0]+x[1]) - d) / (len(x[0]+x[1]))
    if r > .70:
        matches.append([x[0], x[1], d,  round(r*100)]) 
   

In [29]:
df_matches=pd.DataFrame(matches, columns=['x1','x2','distance','ratio'])
likely_matches = df_matches[df_matches.ratio > 90]
likely_matches= likely_matches.drop_duplicates()
print(likely_matches.shape)
likely_matches

(2, 4)


,x1,x2,distance,ratio
0,john smith,john smith,0,100
1,john smith,john smitt,1,95


In [10]:
def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    #print (matrix)
    return (matrix[size_x - 1, size_y - 1])

levenshtein("juan","juan c")

2.0

2.0

In [9]:
'''Levenshtein distance'''
 
from itertools import (accumulate, chain, islice)
from functools import reduce
# levenshtein :: String -> String -> Int


def levenshtein(sa):
    '''Levenshtein distance between
       two strings.'''
    s1 = list(sa)
 
    # go :: [Int] -> Char -> [Int]
    def go(ns, c):
        n, ns1 = ns[0], ns[1:]
 
        # gap :: Int -> (Char, Int, Int) -> Int
        def gap(z, c1xy):
            c1, x, y = c1xy
            return min(
                succ(y),
                succ(z),
                succ(x) if c != c1 else x
            )
        return scanl(gap)(succ(n))(
            zip(s1, ns, ns1)
        )
    return lambda sb: reduce(
        go, list(sb), enumFromTo(0)(len(s1))
    )[-1]
 
 
# TEST ----------------------------------------------------
# main :: IO ()
def main():
    '''Tests'''
 
    pairs = [
        ('rosettacode', 'raisethysword'),
        ('kitten', 'sitting'),
        ('saturday', 'sunday')
    ]
 
    print(
        tabulated(
            'Levenshtein minimum edit distances:\n'
        )(str)(str)(
            uncurry(levenshtein)
        )(concat(map(
            list,
            zip(pairs, map(swap, pairs))
        )))
    )
 
 
# GENERIC -------------------------------------------------
 
# compose (<<<) :: (b -> c) -> (a -> b) -> a -> c
def compose(g):
    '''Right to left function composition.'''
    return lambda f: lambda x: g(f(x))
 
 
# concat :: [[a]] -> [a]
# concat :: [String] -> String
def concat(xxs):
    '''The concatenation of all the elements in a list.'''
    xs = list(chain.from_iterable(xxs))
    unit = '' if isinstance(xs, str) else []
    return unit if not xs else (
        ''.join(xs) if isinstance(xs[0], str) else xs
    )
 
 
# enumFromTo :: (Int, Int) -> [Int]
def enumFromTo(m):
    '''Integer enumeration from m to n.'''
    return lambda n: list(range(m, 1 + n))
 
 
# scanl :: (b -> a -> b) -> b -> [a] -> [b]
def scanl(f):
    '''scanl is like reduce, but returns a succession of
       intermediate values, building from the left.'''
    return lambda a: lambda xs: (
        list(accumulate(chain([a], xs), f))
    )
 
 
# swap :: (a, b) -> (b, a)
def swap(tpl):
    '''The swapped components of a pair.'''
    return (tpl[1], tpl[0])
 
 
# succ :: Int => Int -> Int
def succ(x):
    '''The successor of a value.
       For numeric types, (1 +).'''
    return 1 + x
 
 
# tabulated :: String -> (a -> String) ->
#                        (b -> String) ->
#                        (a -> b) -> [a] -> String
def tabulated(s):
    '''Heading -> x display function ->
                 fx display function ->
                 f -> value list -> tabular string.'''
    def go(xShow, fxShow, f, xs):
        w = max(map(compose(len)(xShow), xs))
        return s + '\n' + '\n'.join([
            xShow(x).rjust(w, ' ') + ' -> ' + fxShow(f(x))
            for x in xs
 
        ])
    return lambda xShow: lambda fxShow: (
        lambda f: lambda xs: go(
            xShow, fxShow, f, xs
        )
    )
 
 
# take :: Int -> [a] -> [a]
# take :: Int -> String -> String
def take(n):
    '''The prefix of xs of length n,
       or xs itself if n > length xs.'''
    return lambda xs: (
        xs[0:n]
        if isinstance(xs, list)
        else list(islice(xs, n))
    )
 
 
# uncurry :: (a -> b -> c) -> ((a, b) -> c)
def uncurry(f):
    '''A function over a tuple
       derived from a curried function.'''
    return lambda xy: f(xy[0])(
        xy[1]
    )
 
 
# MAIN ---
if __name__ == '__main__':
    main()

Levenshtein minimum edit distances:

('rosettacode', 'raisethysword') -> 8
('raisethysword', 'rosettacode') -> 8
           ('kitten', 'sitting') -> 3
           ('sitting', 'kitten') -> 3
          ('saturday', 'sunday') -> 3
          ('sunday', 'saturday') -> 3
